---
RUN THIS NOTEBOOK ON GOOGLE COLAB.
---

---
Before running every cell, download the "sample_annotated_sentences.zip" and "semantic_simlex_v0.1.zip" zip folders and the "preprocessed_dataset.txt" on file section (e.g. /content/) of Google Colab from the following link: https://drive.google.com/drive/folders/1g6fgk86USYCxQHK9IGaIz-AHH6uaEVoy?usp=sharing
---

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install wget

2023-11-19 17:02:53.914085: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-19 17:02:53.914148: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-19 17:02:53.914180: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-19 17:02:53.919767: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-19 17:02:54.774041: W tensorflow/compiler/

# Imports

In [6]:
import numpy as np
import gensim
import matplotlib.pyplot as plt
import seaborn as sns
import wget
import spacy
import scipy.stats
from tqdm import tqdm
from nltk.corpus import stopwords
import nltk
import re
from collections import defaultdict
import json
import pandas as pd
import csv

import numpy as np

import zipfile

# Dataset loading

In [ ]:
zip_jsonl_file_path = '/content/sample_annotated_sentences.zip'
extracted_dir = 'extracted_dir'

with zipfile.ZipFile(zip_jsonl_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

In [ ]:
jsonl_file_path = "/content/extracted_dir/500000.jsonl"

data = []

# Open the JSONL file and parse its content line by line
i = 0
with open(jsonl_file_path, 'r') as file:
    for line in tqdm(file):
        # Parse the line as a JSON object and append it to the data list
        json_obj = json.loads(line)
        data.append(json_obj)
        i+=1

534300it [00:18, 29423.59it/s]


In [ ]:
len(data)

534300

In [ ]:
data

[{'text': 'In Hinduism , the 60th birthday of a man is called Sashti poorthi .',
  'annotations': [{'token_span': [5, 6], 'label': 'birthday%1:28:00::'},
   {'token_span': [8, 9], 'label': 'man%1:18:00::'}]},
 {'text': 'In New Zealand , both Metal Box and Second Edition briefly entered the Top 50 Albums Chart .',
  'annotations': [{'token_span': [6, 7], 'label': 'box%1:06:00::'},
   {'token_span': [8, 9], 'label': 'second%5:00:00:ordinal:00'},
   {'token_span': [11, 12], 'label': 'enter%2:33:00::'},
   {'token_span': [13, 14], 'label': 'top%3:00:00::'}]},
 {'text': 'Article with extensive bibliography Article about the book " L\'hymnaire d\'Adonis " of Jacques d\'Adelswärd - Fersen ( in Spanish ) Pictures of Villa Lysis today [ 1 ] [ 2 ]',
  'annotations': [{'token_span': [0, 1], 'label': 'article%1:10:00::'},
   {'token_span': [4, 5], 'label': 'article%1:10:00::'},
   {'token_span': [7, 8], 'label': 'book%1:10:00::'}]},
 {'text': 'The root cluster attached to the basal plate of the bu

# Dataset preprocessing

THE PREPROCESSING PHASE TAKES ABOUT 90 MINUTES TO BE COMPLETED. IF YOU WANT TO SKIP THIS PART, YOU CAN DIRECTLY USE THE "preprocessed_dataset.txt" IN WHICH I SAVED THE PREPROCESSED DATASET. IF YOU WANT SO, SET THE FOLLOWING VARIABLE (e.g preprocessing variable) False, OTHERWISE True

In [2]:
preprocessing =  False # True if you want to do the preprocessign phase

Here there is a modification of the token spans of each element of the jsonl file by considering as label position not the relative corresponding word position (e.g. the position of the word in the sentence), but the absolute one (e.g. the position of the word in the whole document)

In [ ]:
if preprocessing == True:

  sum_length_previous_sentences = 0
  for j in range(0, len(data)):
      if j == 0:
          continue
      words = data[j-1]['text'].split()
      length_previous_sentences = len(words)
      sum_length_previous_sentences += length_previous_sentences
      for k in range(0, len(data[j]['annotations'])):
          data[j]['annotations'][k]['token_span'][0] += sum_length_previous_sentences
          data[j]['annotations'][k]['token_span'][1] += sum_length_previous_sentences

  data

stopwords

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

The sentences are turned into lower-case, the stopwords and the nonalphanumeric words are excluded and words are substituted with the corresponding sense

In [ ]:
nlp = spacy.load("en_core_web_sm")
# nlp.max_length = 4000000  # Set a higher max_length value
regexp_alphbetic = re.compile('[^a-zA-Z]+')

def preprocess_text(sentence, stopwords, count, lemmatize=True):
  doc = nlp(sentence['text'])
  sentence_tokens = []
  for token in doc:
    token_text = token.lemma_ if lemmatize else token.text
    token_text = token_text.lower()

    # sense substitution
    flag = False
    for j in range(0, len(sentence['annotations'])):
      if count == sentence['annotations'][j]['token_span'][0]:
        token_text = sentence['annotations'][j]['label']
        sentence_tokens.append(token_text)
        count += 1
        flag = True

    if flag == False:
      if token_text in stopwords or regexp_alphbetic.search(token_text):
        count += 1
        continue
      sentence_tokens.append(token_text)
      count += 1

  return sentence_tokens

In [ ]:
if preprocessing == True:

  # remove common words and tokenize
  texts = []
  count = 0
  for i in tqdm(range(0, len(data))):
    # texts.append(preprocess_text(data[i]['text'], stop_words, lemmatize=True))
    texts.append(preprocess_text(data[i], stop_words, count, lemmatize=True))
    count += len(data[i]['text'].split())

    texts

Remove words whose frequency is one

In [ ]:
if preprocessing == True:

  frequency = defaultdict(int)
  for text in tqdm(texts):
      for token in text:
          frequency[token] += 1

  texts = [[token for token in text if frequency[token] > 1]
          for text in tqdm(texts)]



This is to skip the preprocessing phase by using the txt file in which I saved the output of the first preprocessing

In [7]:
# this is to skip the preprocessing phase by using the txt file in which I saved the output of the first preprocessing

if preprocessing == False:

  txt_file_path = "/content/preprocessed_dataset.txt"

  texts = []

  with open(txt_file_path, 'r') as file:
      for line in tqdm(file.readlines()):
          words = line.strip()
          words = words.split()

          arr = []
          for word in words:
              arr.append(word)
          texts.append(arr)

100%|██████████| 534300/534300 [00:07<00:00, 72490.05it/s]


In [9]:
texts

[['hinduism', 'birthday%1:28:00::', 'man%1:18:00::', 'call'],
 ['new',
  'zealand',
  'metal',
  'box%1:06:00::',
  'second%5:00:00:ordinal:00',
  'edition',
  'briefly',
  'enter%2:33:00::',
  'top%3:00:00::',
  'albums',
  'chart'],
 ['article%1:10:00::',
  'extensive',
  'bibliography',
  'article%1:10:00::',
  'book%1:10:00::',
  'jacques',
  'fersen',
  'spanish',
  'picture',
  'villa',
  'lysis',
  'today'],
 ['root%1:20:00::',
  'cluster',
  'attach%2:35:02::',
  'basal',
  'plate%1:06:02::',
  'bulb%1:20:00::',
  'part',
  'typically',
  'consider',
  'palatable',
  'form'],
 ['world%1:14:01::',
  'world',
  'word%1:10:00::',
  'word',
  'come',
  'dictionary%1:10:00::',
  'dictionary',
  'word%1:10:00::',
  'word',
  'prepare',
  'nathan',
  'bailey',
  'though',
  'none',
  'comprehensive',
  'breadth',
  'style%1:10:00::',
  'style',
  'johnson'],
 ['garlic',
  'soften',
  'extract',
  'clove',
  'squeeze',
  'root%1:20:00::',
  'end',
  'bulb%1:20:00::',
  'individually',


# Model

In [ ]:
model = gensim.models.Word2Vec(texts, vector_size=10, window=2, min_count=1)
model

In [ ]:
model.wv["albums"]

array([ 2.4258416 , -1.2336053 ,  1.0260677 , -2.0056756 , -0.2428784 ,
        0.9066873 ,  0.49862084,  0.24137428, -1.1229185 , -0.68120587],
      dtype=float32)

In [ ]:
model.wv.most_similar('albums')

[('hits', 0.9548279047012329),
 ('aflw', 0.9543217420578003),
 ('singles', 0.9510210752487183),
 ('airplay', 0.9470522999763489),
 ('film', 0.9449800848960876),
 ('greatest', 0.9428974390029907),
 ('mtv', 0.9428024888038635),
 ('mainstreet', 0.9402492642402649),
 ('best', 0.9402034282684326),
 ('critics', 0.9382091760635376)]

# Non-semantic

The results are written in the non_semantic.tsv file directly from the following function

In [ ]:
tsv_file_path = "/content/non_semantic.tsv"

def compute_correlation_score(model, word_pair2score, print_warning=True):
  human_scores = []
  system_scores = []
  with open(tsv_file_path, 'w', newline='') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')  # Use a tab as the delimiter
    for (w1, w2), score in word_pair2score.items():
      if (w1 not in model) or (w2 not in model):
        system_scores.append(-1)
        writer.writerows([[w1, w2, -1]])
        human_scores.append(score)
        if print_warning:
          print(f"WARNING ({w1} and {w2}) are not present in the embedding model!!" )
        continue
      system_similarity = model.similarity(w1, w2)
      writer.writerows([[w1, w2, system_similarity]])

      # Write the data to the TSV file

      human_scores.append(score)
      system_scores.append(system_similarity)

    human_scores = np.array(human_scores)
    system_scores = np.array(system_scores)
    pearson_r, _ = scipy.stats.pearsonr(human_scores, system_scores)    # Pearson's r
    spearman_rho = scipy.stats.spearmanr(human_scores, system_scores).correlation   # Spearman's rho
    return pearson_r, spearman_rho

In [ ]:
zip_tsv_file_path = '/content/semantic_simlex_v0.1.zip'
extracted_dir = 'extracted_dir'

with zipfile.ZipFile(zip_tsv_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

In [ ]:
tsv_file_path_new = "/content/extracted_dir/semantic_simlex_v0.1.tsv"

non_semantic_simplex_pairs = dict()

data = pd.read_csv(tsv_file_path_new, sep='\t')

# print(data)

for index, row in data.iterrows():
    non_semantic_simplex_pairs[(row['word1'], row['word2'])] = float(row['SimLex999'])



In [ ]:
non_semantic_simplex_pairs

{('glass', 'crystal'): 6.27,
 ('task', 'woman'): 0.68,
 ('house', 'barn'): 4.33,
 ('vision', 'perception'): 6.88,
 ('box', 'hat'): 1.3,
 ('metal', 'aluminum'): 7.25,
 ('doctor', 'orthodontist'): 5.58,
 ('letter', 'paragraph'): 4.0,
 ('man', 'child'): 4.13,
 ('home', 'state'): 2.58,
 ('mink', 'fur'): 6.83,
 ('mother', 'wife'): 3.02,
 ('child', 'adult'): 2.98,
 ('fee', 'payment'): 7.15,
 ('milk', 'juice'): 4.05,
 ('cup', 'cone'): 3.17,
 ('book', 'article'): 5.43,
 ('money', 'capital'): 6.67,
 ('body', 'shoulder'): 2.88,
 ('cheek', 'tongue'): 4.52,
 ('father', 'brother'): 4.2,
 ('book', 'information'): 5.0,
 ('appointment', 'engagement'): 6.75,
 ('gun', 'knife'): 3.65,
 ('meat', 'sandwich'): 2.35,
 ('birthday', 'year'): 1.67,
 ('formula', 'equation'): 7.95,
 ('wire', 'cord'): 7.62,
 ('car', 'hose'): 0.87,
 ('room', 'bath'): 3.33,
 ('rice', 'bean'): 2.72,
 ('crime', 'fraud'): 5.65,
 ('disease', 'infection'): 7.15,
 ('winner', 'presence'): 1.08,
 ('atom', 'carbon'): 3.1,
 ('shore', 'coast')

In [ ]:
compute_correlation_score(model.wv, non_semantic_simplex_pairs, print_warning=False)

(0.23726584795133082, 0.216862430874881)

# Semantic

In [ ]:
tsv_file_path = "/content/extracted_dir/semantic_simlex_v0.1.tsv"

words_dict = dict()

semantic_simplex_pairs = dict()

def get_semantix_simplex_pairs(path):
  semantic_simplex_pairs = dict()
  with open(path) as fr:
    next(fr)
    for line in fr:
      chunks = line.strip().split()
      w1 = chunks[0]
      w2 = chunks[1]
      score = chunks[3]
      senses_1 = chunks[10].split(",")
      senses_2 = chunks[11].split(",")
      semantic_simplex_pairs[(tuple(senses_1), tuple(senses_2))] = score
      words_dict[tuple(senses_1)] = w1
      words_dict[tuple(senses_2)] = w2

  return semantic_simplex_pairs

In [ ]:
semantic_simplex_pairs =  get_semantix_simplex_pairs(tsv_file_path)
semantic_simplex_pairs

{(('glass%1:27:00::',), ('crystal%1:27:00::',)): '6.27',
 (('task%1:04:01::',), ('woman%1:18:00::',)): '0.68',
 (('house%1:06:00::',), ('barn%1:06:00::',)): '4.33',
 (('vision%1:09:01::',), ('perception%1:09:00::',)): '6.88',
 (('box%1:06:00::',), ('hat%1:06:00::',)): '1.3',
 (('metal%1:27:00::', 'metal%1:27:01::'), ('aluminum%1:27:00::',)): '7.25',
 (('doctor%1:18:00::',), ('orthodontist%1:18:00::',)): '5.58',
 (('letter%1:10:00::',), ('paragraph%1:10:00::',)): '4',
 (('man%1:18:00::',), ('child%1:18:00::',)): '4.13',
 (('home%1:15:02::',), ('state%1:15:01::',)): '2.58',
 (('mink%1:27:00::',), ('fur%1:27:00::',)): '6.83',
 (('mother%1:18:00::',), ('wife%1:18:00::',)): '3.02',
 (('child%1:18:00::',), ('adult%1:18:00::',)): '2.98',
 (('fee%1:21:00::',), ('payment%1:21:00::',)): '7.15',
 (('milk%1:13:01::',), ('juice%1:13:00::',)): '4.05',
 (('cup%1:25:00::',), ('cone%1:06:00::',)): '3.17',
 (('book%1:10:00::',), ('article%1:10:00::',)): '5.43',
 (('money%1:21:02::',), ('capital%1:21:00:

In [ ]:
words_dict

{('glass%1:27:00::',): 'glass',
 ('crystal%1:27:00::',): 'crystal',
 ('task%1:04:01::',): 'task',
 ('woman%1:18:00::',): 'woman',
 ('house%1:06:00::',): 'house',
 ('barn%1:06:00::',): 'barn',
 ('vision%1:09:01::',): 'vision',
 ('perception%1:09:00::',): 'perception',
 ('box%1:06:00::',): 'box',
 ('hat%1:06:00::',): 'hat',
 ('metal%1:27:00::', 'metal%1:27:01::'): 'metal',
 ('aluminum%1:27:00::',): 'aluminum',
 ('doctor%1:18:00::',): 'doctor',
 ('orthodontist%1:18:00::',): 'orthodontist',
 ('letter%1:10:00::',): 'letter',
 ('paragraph%1:10:00::',): 'paragraph',
 ('man%1:18:00::',): 'man',
 ('child%1:18:00::',): 'child',
 ('home%1:15:02::',): 'home',
 ('state%1:15:01::',): 'state',
 ('mink%1:27:00::',): 'mink',
 ('fur%1:27:00::',): 'fur',
 ('mother%1:18:00::',): 'mother',
 ('wife%1:18:00::',): 'wife',
 ('adult%1:18:00::',): 'adult',
 ('fee%1:21:00::',): 'fee',
 ('payment%1:21:00::',): 'payment',
 ('milk%1:13:01::',): 'milk',
 ('juice%1:13:00::',): 'juice',
 ('cup%1:25:00::',): 'cup',
 ('c

The results are written in the semantic.tsv file directly from the following function

In [ ]:
tsv_file_path = "/content/semantic.tsv"



def compute_semantic_correlation_score(model, senses2score,  print_warning=True):
  human_scores = []
  system_scores = []
  with open(tsv_file_path, 'w', newline='') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')  # Use the tab as the delimiter
    for (senses_1, senses_2), score in senses2score.items():
      human_scores.append(score)
      senses_1_in_model = [s for s in senses_1 if s in model]
      senses_2_in_model = [s for s in senses_2 if s in model]

      if len(senses_1_in_model) == 0 or len(senses_2_in_model) == 0:
        # sense is not present in the model
        s1_str = " ".join(senses_1)
        s2_str = " ".join(senses_2)
        if print_warning:
          print(f"WARNING ({s1_str} and {s2_str}) are not present in the embedding model!!" )
        system_scores.append(-1)
        writer.writerows([[words_dict[senses_1], words_dict[senses_2], -1]])
      # Calculate semantic similarities between all pairs of senses
      else:
        all_similarities = []
        for s1 in senses_1_in_model:
          for s2 in senses_2_in_model:
            all_similarities.append(model.similarity(s1, s2))

        system_similarity = np.max(all_similarities)

        # system_similarity = sum(all_similarities) / len(all_similarities)
        # human_scores.append(score)
        writer.writerows([[words_dict[senses_1], words_dict[senses_2], system_similarity]])
        system_scores.append(system_similarity)


    human_scores = np.array(human_scores, dtype='float64')
    system_scores = np.array(system_scores)
    # Calculate Pearson's r (Pearson correlation coefficient) and Spearman's rho (Spearman rank correlation coefficient)
    pearson_r, _ = scipy.stats.pearsonr(human_scores, system_scores)    # Pearson's r
    spearman_rho = scipy.stats.spearmanr(human_scores, system_scores).correlation   # Spearman's rho
    return pearson_r, spearman_rho



In [ ]:
compute_semantic_correlation_score(model.wv, semantic_simplex_pairs, print_warning=False)

(0.21116090026498222, 0.2019257423825855)

# Explicit Representation

To implement this part, I used suggestions at this link: https://stackoverflow.com/questions/58701337/how-to-construct-ppmi-matrix-from-a-text-corpus

In [ ]:
# This function returns a nxn squared matrix (where n  is the number of the preprocessed words/senses) whose each cell (i,j) contains 1 if word/sense i and word/sense j
# co-occur in the same sentence, 0 otherwise

def co_occurrence(sentences, window_size):
    d = defaultdict(int)
    vocab = set()
    for words in tqdm(sentences):
        for i in range(len(words)):
            word = words[i]
            vocab.add(word)
            next_word = words[i+1 : i+1+window_size]
            for t in next_word:
                key = tuple( sorted([t, word]) )
                d[key] += 1

    # transform the dictionary into dataframe
    vocab = sorted(vocab)
    df = pd.DataFrame(data=np.zeros((len(vocab), len(vocab)), dtype=np.int16),
                      index=vocab,
                      columns=vocab)
    for key, value in tqdm(d.items()):
        df.at[key[0], key[1]] = value
        df.at[key[1], key[0]] = value
    return df

In [ ]:
df = co_occurrence(texts[0: 10], 5) # only the first 10 samples are taken for memory reasons
df

100%|██████████| 481/481 [00:00<00:00, 9362.24it/s]


,albums,alcoholic,article%1:10:00::,attach%2:35:02::,attalea,attraction,bailey,basal,beach%1:17:00::,beverage,...,vehicle%1:06:00::,version,villa,wave,well,word,word%1:10:00::,world,world%1:14:01::,zealand
albums,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
alcoholic,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
article%1:10:00::,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
attach%2:35:02::,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
attalea,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
word,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,3,1,1,0
word%1:10:00::,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,3,1,1,1,0
world,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0
world%1:14:01::,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,0,0


In [ ]:
# This function returns a squared matrix whose each cell (i,j) contains the ppmi value between word/sense i and word/sense j

def pmi_matrix(df, positive=True):
    col_totals = df.sum(axis=0)
    total = col_totals.sum()
    row_totals = df.sum(axis=1)
    expected = np.outer(row_totals, col_totals) / total
    df = df / expected
    with np.errstate(divide='ignore'):
        df = np.log(df)
    df[np.isinf(df)] = 0.0
    if positive:
        df[df < 0] = 0.0
    return df

In [ ]:
ppmi_matrix = pmi_matrix(df, positive=True)


In [ ]:
ppmi_matrix

,albums,alcoholic,article%1:10:00::,attach%2:35:02::,attalea,attraction,bailey,basal,beach%1:17:00::,beverage,...,vehicle%1:06:00::,version,villa,wave,well,word,word%1:10:00::,world,world%1:14:01::,zealand
albums,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
alcoholic,0.0,0.000000,0.000000,0.0,2.755848,0.0,0.000000,0.000000,0.0,2.638065,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
article%1:10:00::,0.0,0.000000,1.944918,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
attach%2:35:02::,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,2.889379,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
attalea,0.0,2.755848,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,2.638065,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
word,0.0,0.000000,0.000000,0.0,0.000000,0.0,1.778932,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.248304,2.407541,2.289758,2.472080,0.0
word%1:10:00::,0.0,0.000000,0.000000,0.0,0.000000,0.0,1.839557,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,2.407541,1.369553,2.350383,2.532704,0.0
world,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,2.289758,2.350383,0.000000,3.513534,0.0
world%1:14:01::,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,2.472080,2.532704,3.513534,0.000000,0.0


In [ ]:
# we are getting the rows that contain at least one value different from zero
non_zero_rows = ppmi_matrix.index[ppmi_matrix.any(axis=1)].tolist()

# we select randomly an index
index = np.random.choice(non_zero_rows)

# get the columns
row = ppmi_matrix.loc[index]

# Filter out columns with only zero values
non_zero_cols = row[row > 0].sort_values(ascending=False)


# results
print('Word: ', row.name)
print('ppmi values: \n')
for col in non_zero_cols.index:
    print(f"\t {col} = {row[col]}")

Word:  burst
ppmi values: 

	 bomb = 3.225851438604626
	 useful = 3.043529881810671
	 high = 2.638064773702507
	 confine = 2.5327042580446806
	 container = 2.5327042580446806
	 container%1:06:00:: = 2.5327042580446806
	 quantity = 2.5327042580446806
	 pressure = 2.0020760069825103
